# Azure Quantum Backend

## Prerequisites

- An Azure account with active subcription.
- An Azure Quantum workspace. ([How to create this?](https://docs.microsoft.com/en-us/azure/quantum/how-to-create-workspace?tabs=tabid-quick))
- Resource ID and location of Azure Quantum workspace.
- Install Azure Quantum dependencies for ProjectQ.
```
python -m pip install --user projectq[azure-quantum]
```

## Load Imports

Run following cell to load requried imports

In [ ]:
import matplotlib.pyplot as plt

from projectq import MainEngine
from projectq.ops import H, CX, All, Measure
from projectq.cengines import BasicMapperEngine
from projectq.backends import AzureQuantumBackend
from projectq.libs.hist import histogram

## Initialize Azure Quantum backend

Update `resource_id` and `location` of your Azure Quantum workspace in below cell and run to initialize Azure Quantum workspace.

Following are valid `target_names`:
- ionq.simulator
- ionq.qpu
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s1

Flag `use_hardware` represents wheather or not use real hardware or just a simulator. If False regardless of the value of `target_name`, simulator will be used.

In [ ]:
azure_quantum_backend = AzureQuantumBackend(
    use_hardware=False,
    target_name='ionq.simulator',
    resource_id="<resource-id>",  # resource id of workspace
    location="<location>",  # location of workspace
    verbose=True
)

## Create ProjectQ Engine

Initialize ProjectQ `MainEngine` using Azure Quantum as backend.

In [ ]:
mapper = BasicMapperEngine()
max_qubits = 3

mapping = {}
for i in range(max_qubits):
    mapping[i] = i

mapper.current_mapping = mapping

main_engine = MainEngine(
    backend=azure_quantum_backend,
    engine_list=[mapper]
)

## Create circuit using ProjectQ lean syntax!

Allocate qubits, build circuit and measure qubits.

In [ ]:
circuit = main_engine.allocate_qureg(3)
q0, q1, q2 = circuit

H | q0
CX | (q0, q1)
CX | (q1, q2)
All(Measure) | circuit

## Run circuit and get result

Flush down circuit to Azure Quantum backend and wait for results. It prints `job-id` for the reference (in case this operation timed-out). 

In [ ]:
main_engine.flush()

print(azure_quantum_backend.get_probabilities(circuit))

## Timed out! Re-run the circuit with retrieve_execution argument

If job execution timed-out, use `retrieve_execution` argument retrive result instead of re-running the circuit. Use `job-id` from previous cell.

In [ ]:
azure_quantum_backend = AzureQuantumBackend(
    use_hardware=False,
    target_name='ionq.simulator',
    resource_id="<resource-id>",  # resource id of workspace
    location="<location>",  # location of workspace
    retrieve_execution="<job-id>",  # job-id of Azure Quantum job
    verbose=True
)

mapper = BasicMapperEngine()
max_qubits = 10

mapping = {}
for i in range(max_qubits):
    mapping[i] = i

mapper.current_mapping = mapping

main_engine = MainEngine(
    backend=azure_quantum_backend,
    engine_list=[mapper]
)

circuit = main_engine.allocate_qureg(3)
q0, q1, q2 = circuit

H | q0
CX | (q0, q1)
CX | (q1, q2)
All(Measure) | circuit

main_engine.flush()

print(azure_quantum_backend.get_probabilities(circuit))

# Plot Histogram

Now, let's plot histogram with above result.

In [ ]:
histogram(main_engine.backend, circuit)
plt.show()